In [216]:
!pip -q install pyfiglet
from pyfiglet import Figlet
f = Figlet(font='larry3d')     # prueba: 'slant','standard','banner3-D','larry3d'
print(f.renderText('FARMACIA PEREZ'))


 ____    ______  ____             ______  ____    ______   ______     
/\  _`\ /\  _  \/\  _`\   /'\_/`\/\  _  \/\  _`\ /\__  _\ /\  _  \    
\ \ \L\_\ \ \L\ \ \ \L\ \/\      \ \ \L\ \ \ \/\_\/_/\ \/ \ \ \L\ \   
 \ \  _\/\ \  __ \ \ ,  /\ \ \__\ \ \  __ \ \ \/_/_ \ \ \  \ \  __ \  
  \ \ \/  \ \ \/\ \ \ \\ \\ \ \_/\ \ \ \/\ \ \ \L\ \ \_\ \__\ \ \/\ \ 
   \ \_\   \ \_\ \_\ \_\ \_\ \_\\ \_\ \_\ \_\ \____/ /\_____\\ \_\ \_\
    \/_/    \/_/\/_/\/_/\/ /\/_/ \/_/\/_/\/_/\/___/  \/_____/ \/_/\/_/
                                                                      
                                                                      
 ____    ____    ____    ____    ________     
/\  _`\ /\  _`\ /\  _`\ /\  _`\ /\_____  \    
\ \ \L\ \ \ \L\_\ \ \L\ \ \ \L\_\/____//'/'   
 \ \ ,__/\ \  _\L\ \ ,  /\ \  _\L    //'/'    
  \ \ \/  \ \ \L\ \ \ \\ \\ \ \L\ \ //'/'___  
   \ \_\   \ \____/\ \_\ \_\ \____/ /\_______\
    \/_/    \/___/  \/_/\/ /\/___/  \/_______/
                                


## Cotizaciones

En este código podrás subir tus archivos de cotización en formato **.xlsx** para comparar los precios entre distintos proveedores y encontrar las mejores opciones.

Para usarlo:

1. Da clic en “Ejecutar todo” en el menú superior.

2. En la segunda celda, selecciona los archivos correspondientes a cada proveedor.

⚠️ **Importante:** Cada archivo debe incluir las siguientes columnas (con exactamente estos nombres):

- SKU

- Nombre

- Precio Unitario


In [217]:
from google.colab import files
import pandas as pd
import os
import glob
import datetime
import pytz
from IPython.display import display, HTML

## Segunda Celda
Aqui agrega tus archivos

In [218]:
# 🧹 Eliminar cualquier archivo .xlsx previo en el directorio actual
for f in glob.glob("*.xlsx"):
    os.remove(f)

# 📤 Subir nuevos archivos Excel (puedes seleccionar varios)
print("📤 Sube tus archivos Excel (uno por proveedor)...")
uploaded = files.upload()

📤 Sube tus archivos Excel (uno por proveedor)...


Saving dummy_productos_3.xlsx to dummy_productos_3.xlsx
Saving dummy_productos_2.xlsx to dummy_productos_2.xlsx
Saving dummy_productos_1.xlsx to dummy_productos_1.xlsx


In [219]:
# 📚 Leer y procesar cada archivo subido
dataframes = {}
for filename in uploaded.keys():
    try:
        df = pd.read_excel(filename)
        # Normaliza encabezados y columnas clave
        df.columns = df.columns.str.strip()

        # Asegura existencia de columnas mínimas (SKU y Precio Unitario). 'Nombre' puede faltar.
        min_cols = ['SKU', 'Precio Unitario']
        if not all(col in df.columns for col in min_cols):
            print(f"⚠️ {filename} no tiene las columnas mínimas {min_cols}. Se omite.")
            continue
        if 'Nombre' not in df.columns:
            df['Nombre'] = ""  # Nombre opcional; no vamos a depender de él

        # 🏷️ Añadir proveedor (ya lo haces más abajo, lo dejo aquí por orden)
        df['Proveedor'] = filename.replace('.xlsx', '').replace('.xls', '')

        # Normaliza SKU como texto limpio
        df['SKU'] = df['SKU'].astype(str).str.strip()
        # Detectar SKUs inválidos: letras, puntos decimales o vacíos
        invalid_skus = df[~df['SKU'].str.match(r'^\d+$', na=False)]

        if not invalid_skus.empty:
            print(f"❌ ERROR: En el archivo '{filename}' se detectaron SKUs inválidos:\n")
            display(invalid_skus[['SKU']])
            raise ValueError(
                f"El archivo '{filename}' contiene SKUs no válidos (con letras, decimales o vacíos). Corrige y vuelve a intentar."
            )

        # Si todo OK, convertir SKU a texto limpio y uniforme
        df['SKU'] = df['SKU'].astype(str)

        # Limpia Nombre (puede llegar distinto por archivo)
        df['Nombre'] = df['Nombre'].astype(str).str.strip()

        # Limpia Precio Unitario: quita símbolos, comas y espacios antes de convertir a número
        df['Precio Unitario'] = (
            df['Precio Unitario']
              .astype(str)
              .str.replace(r'[^0-9,\.\-]', '', regex=True)  # quita $, espacios, etc.
              .str.replace(',', '', regex=False)           # quita separador de miles
        )
        df['Precio Unitario'] = pd.to_numeric(df['Precio Unitario'], errors='coerce')


        # 🧩 Verificar que las columnas esperadas existan
        expected_cols = ['SKU', 'Precio Unitario']  # 'Nombre' ya no es obligatorio
        if not all(col in df.columns for col in expected_cols):
            print(f"⚠️ {filename} no tiene las columnas esperadas. Se omite.")
            continue

        # 🏷️ Añadir columna con el nombre del proveedor
        df['Proveedor'] = filename.replace('.xlsx', '').replace('.xls', '')

        # 🧽 Limpiar nombres de productos y tipos de datos
        df['Nombre'] = df['Nombre'].astype(str).str.strip()
        df['Precio Unitario'] = pd.to_numeric(df['Precio Unitario'], errors='coerce')

        # Guardar en diccionario
        dataframes[filename] = df
        print(f"✅ Archivo cargado correctamente: {filename}")

    except Exception as e:
      print(f"❌ Error al leer {filename}: {e}")
      raise  # 🚨 vuelve a lanzar el error y detiene todo

✅ Archivo cargado correctamente: dummy_productos_3.xlsx
✅ Archivo cargado correctamente: dummy_productos_2.xlsx
✅ Archivo cargado correctamente: dummy_productos_1.xlsx


In [220]:
# 👀 Mostrar los DataFrames cargados
for nombre, df in dataframes.items():
    print(f"\n📄 {nombre}:")
    display(df)


📄 dummy_productos_3.xlsx:


,SKU,Nombre,Precio Unitario,Proveedor
0,1,Producto A c/8,121.0000,dummy_productos_3
1,2,Producto B,76.0000,dummy_productos_3
2,3,Producto C,151.0000,dummy_productos_3
3,4,Producto D,152.0000,dummy_productos_3
4,5,Producto E con 8,24.0000,dummy_productos_3
5,6,Producto F con 10 tab,68.0000,dummy_productos_3
6,9,torrilla,12.1234,dummy_productos_3



📄 dummy_productos_2.xlsx:


,Nombre,SKU,Precio Unitario,Unidades,Proveedor
0,Producto A,1,99.9900,10 t,dummy_productos_2
1,Producto B,2,130.0000,20 c,dummy_productos_2
2,Producto C,3,151.2456,10 s,dummy_productos_2
3,Producto E,5,NaN,14 t,dummy_productos_2
4,Producto G,7,56.0000,30 t,dummy_productos_2
5,torrilla,9,12.1233,NaN,dummy_productos_2



📄 dummy_productos_1.xlsx:


,SKU,Nombre,Precio Unitario,Proveedor
0,1,Producto A,120.500,dummy_productos_1
1,2,Producto B,75.000,dummy_productos_1
2,3,Producto C,151.000,dummy_productos_1
3,4,Producto D,151.000,dummy_productos_1
4,5,Producto E,23.000,dummy_productos_1
5,6,Producto F,67.089,dummy_productos_1


In [221]:
# 📊 Unir todos los proveedores en un solo DataFrame
merged_df = pd.concat(dataframes.values(), ignore_index=True)

# =========================
# 🧠 Nombre canónico por SKU
# =========================
# Estrategia: usar el nombre más frecuente (mode). Si no hay mode claro, usa el más largo.
nombres_canon = (
    merged_df.assign(Nombre_norm=merged_df['Nombre'].astype(str).str.strip())
             .groupby('SKU')['Nombre_norm']
             .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else max(s, key=len) if len(s) else "")
             .rename('Nombre_canonico')
             .reset_index()
)

# Anexar el nombre canónico a la tabla combinada
merged_df = merged_df.merge(nombres_canon, on='SKU', how='left')

# 🔢 Ordenar por SKU (de menor a mayor)
merged_df = merged_df.sort_values(by='SKU', ascending=True)

# 🔄 Reindexar después del ordenamiento (opcional, solo para que los índices queden limpios)
merged_df.reset_index(drop=True, inplace=True)

print("\n📊 Vista general combinada y ordenada por SKU:")
display(merged_df.head(20))  # muestra las primeras 20 filas


📊 Vista general combinada y ordenada por SKU:


,SKU,Nombre,Precio Unitario,Proveedor,Unidades,Nombre_canonico
0,1,Producto A c/8,121.0000,dummy_productos_3,NaN,Producto A
1,1,Producto A,120.5000,dummy_productos_1,NaN,Producto A
2,1,Producto A,99.9900,dummy_productos_2,10 t,Producto A
3,2,Producto B,76.0000,dummy_productos_3,NaN,Producto B
4,2,Producto B,75.0000,dummy_productos_1,NaN,Producto B
5,2,Producto B,130.0000,dummy_productos_2,20 c,Producto B
6,3,Producto C,151.0000,dummy_productos_1,NaN,Producto C
7,3,Producto C,151.2456,dummy_productos_2,10 s,Producto C
8,3,Producto C,151.0000,dummy_productos_3,NaN,Producto C
9,4,Producto D,152.0000,dummy_productos_3,NaN,Producto D


In [222]:
# 🔢 Ignorar precios no válidos
tmp = merged_df.dropna(subset=['Precio Unitario']).copy()

# 💰 Redondear a 4 decimales solo para COMPARACIÓN
# (mantiene los valores originales para mostrar)
tmp['Precio_cmp'] = tmp['Precio Unitario'].astype(float).round(4)

# 🔍 Calcular precio mínimo redondeado por SKU
min_por_sku = tmp.groupby('SKU')['Precio_cmp'].transform('min')

# 🔝 Mejor precio por SKU (uno solo)
idx = tmp.groupby('SKU')['Precio_cmp'].idxmin()
mejores_precios_df = tmp.loc[idx].sort_values('SKU').reset_index(drop=True)

#print("✅ Proveedor más barato por SKU (idxmin):")
#display(mejores_precios_df)

# ✅ Identificar empates: los que igualan el precio mínimo redondeado
empates_df = (
    tmp[min_por_sku == tmp['Precio_cmp']]
      .sort_values(['SKU', 'Proveedor'])
      .reset_index(drop=True)
)

#print("ℹ️ SKUs con todos los proveedores empatados al precio mínimo (comparando a 4 decimales):")
#display(empates_df)


In [223]:
tmp = merged_df.dropna(subset=['Precio Unitario']).copy()
min_por_sku = tmp.groupby('SKU')['Precio Unitario'].transform('min')
empates_df = (
    tmp[min_por_sku == tmp['Precio Unitario']]
      .sort_values(['SKU', 'Proveedor'])
      .reset_index(drop=True)
)

print("ℹ️ SKUs con todos los proveedores empatados al precio mínimo:")
display(empates_df[['SKU', 'Nombre', 'Precio Unitario']])


ℹ️ SKUs con todos los proveedores empatados al precio mínimo:


,SKU,Nombre,Precio Unitario
0,1,Producto A,99.9900
1,2,Producto B,75.0000
2,3,Producto C,151.0000
3,3,Producto C,151.0000
4,4,Producto D,151.0000
5,5,Producto E,23.0000
6,6,Producto F,67.0890
7,7,Producto G,56.0000
8,9,torrilla,12.1233


In [224]:
# 🕒 Crear timestamp actual (formato legible: YYYYMMDD_HHMMSS)
cst = pytz.timezone("America/Mexico_City")
timestamp = datetime.datetime.now(cst).strftime("%Y%m%d_%H%M%S")

# 📁 Crear carpeta de resultados
os.makedirs("resultados", exist_ok=True)

# 📄 Nombre del archivo con timestamp y palabra 'cotizacion'
filename = f"resultados/cotizacion_{timestamp}.xlsx"

# 💾 Guardar el DataFrame de empates en un archivo Excel
empates_df.to_excel(filename, index=False)

print(f"✅ Archivo generado y guardado en: {filename}")

# 📤 Descargar el archivo (opcional, si estás en Google Colab)
files.download(filename)


✅ Archivo generado y guardado en: resultados/cotizacion_20251025_230813.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [225]:
# =========================
# 📝 Resumen profesional a partir de empates_df
# =========================

# Copia y columnas auxiliares para orden
_emp = empates_df.copy()
_emp['SKU_str'] = _emp['SKU'].astype(str)
_emp['SKU_num'] = pd.to_numeric(_emp['SKU'], errors='coerce')

# Separar ganadores únicos (SKU que aparecen 1 sola vez en empates_df)
counts = _emp.groupby('SKU_str')['SKU_str'].transform('size')
ganadores_unicos = _emp[counts == 1].copy()
empates_reales  = _emp[counts > 1].copy()

# Ordenar para salida limpia
ganadores_unicos = ganadores_unicos.sort_values(
    by=['Proveedor', 'SKU_num', 'SKU_str'], ascending=[True, True, True]
).reset_index(drop=True)

empates_reales = empates_reales.sort_values(
    by=['SKU_num', 'Proveedor', 'Precio Unitario'], ascending=[True, True, True]
).reset_index(drop=True)

# Construir mensaje
lineas = []
lineas.append("Tu cotización ha finalizado. El resumen es:\n")

if ganadores_unicos.empty:
    lineas.append("No hay ganadores únicos por SKU.\n")
else:
    for prov, g in ganadores_unicos.groupby('Proveedor', sort=True):
        lineas.append(f"{prov} — tendrás que pedirle los siguientes productos:")
        for _, r in g.iterrows():
            lineas.append(f"  - SKU {r['SKU']} — {r['Nombre_canonico']} — ${r['Precio Unitario']:.4f}")
        lineas.append("")  # línea en blanco entre proveedores

# Empates al final
lineas.append("A continuación se muestran los empates para que decidas tú a quién pedirlo:")

if empates_reales.empty:
    lineas.append("No hay empates.")
else:
    for sku, g in empates_reales.groupby('SKU_str', sort=False):
        # usa la primera fila para mostrar el nombre
        nombre = g.iloc[0]['Nombre_canonico']
        lineas.append(f"  • SKU {sku} — {nombre}:")
        for _, r in g.iterrows():
            lineas.append(f"      - {r['Proveedor']} — ${r['Precio Unitario']:.4f}")
        lineas.append("")

mensaje_resumen = "\n".join(lineas)
print(mensaje_resumen)


Tu cotización ha finalizado. El resumen es:

dummy_productos_1 — tendrás que pedirle los siguientes productos:
  - SKU 2 — Producto B — $75.0000
  - SKU 4 — Producto D — $151.0000
  - SKU 5 — Producto E — $23.0000
  - SKU 6 — Producto F — $67.0890

dummy_productos_2 — tendrás que pedirle los siguientes productos:
  - SKU 1 — Producto A — $99.9900
  - SKU 7 — Producto G — $56.0000
  - SKU 9 — torrilla — $12.1233

A continuación se muestran los empates para que decidas tú a quién pedirlo:
  • SKU 3 — Producto C:
      - dummy_productos_1 — $151.0000
      - dummy_productos_3 — $151.0000



In [226]:
def _fmt_money4(x):
    try:
        return f"${float(x):,.4f}"
    except:
        return x

# Reinicia el HTML
html_message = """
<div class='box'>
  <h3>💼 Tu cotización ha finalizado</h3>
  <p style='color:#7F8C8D; margin-top:-6px'>
    Resumen profesional a partir de precios mínimos y empates por SKU.
  </p>

  <h4>🏆 Ganadores por proveedor</h4>
"""

# ===== Ganadores por proveedor: tabla por proveedor =====
if not ganadores_unicos.empty:
    for prov, g in ganadores_unicos.groupby('Proveedor', sort=True):
        g2 = (
            g[['SKU','Nombre_canonico','Precio Unitario']]
            .rename(columns={'Nombre_canonico':'Nombre', 'Precio Unitario':'Precio'})
            .sort_values(['SKU'])
            .copy()
        )
        g2['Precio'] = g2['Precio'].map(_fmt_money4)
        html_message += f"<h5>🏪 {prov}</h5>"
        html_message += g2.to_html(index=False, escape=False, classes='tbl')
        html_message += "<br>"
else:
    html_message += "<p style='color:#7F8C8D;'>No hay ganadores únicos por SKU.</p>"

# ===== Empates detectados: una sección por SKU =====
html_message += "<hr><h4>⚖️ Empates detectados</h4>"

if not empates_reales.empty:
    for sku, g in empates_reales.groupby('SKU_str', sort=False):
        nombre = g.iloc[0]['Nombre_canonico']
        html_message += f"<h5>SKU {sku} — {nombre}</h5>"
        e2 = (
            g[['Proveedor','Precio Unitario']]
            .rename(columns={'Precio Unitario':'Precio'})
            .sort_values(['Proveedor'])
            .copy()
        )
        e2['Precio'] = e2['Precio'].map(_fmt_money4)
        html_message += e2.to_html(index=False, escape=False, classes='tbl')
        html_message += "<br>"
else:
    html_message += "<p style='color:#7F8C8D;'>No hay empates.</p>"

# ===== Estilos (mismo formato que vienes usando) =====
html_message += """
</div>

<style>
  /* 🔧 Contenedor general */
  .box {
    font-family: Roboto, Arial, sans-serif;
    font-size: 15px;
    background: #F9FBFD;
    border: 1px solid #D6EAF8;
    border-radius: 10px;
    padding: 16px;
    text-align: left;
  }

  .box h3 {
    color: #2E86C1;
    margin: 0 0 12px 0;
    text-align: left;
  }

  .box h4 {
    color: #34495E;
    margin: 14px 0 8px 0;
    text-align: left;
  }

  .box h5 {
    color: #1A5276;
    margin: 10px 0 6px 0;
    text-align: left;
  }

  /* 🔧 Tabla base */
  .tbl {
    width: auto;
    border-collapse: collapse;
    background: #fff;
    border: 1px solid #E5EAF2;
    border-radius: 10px;
    margin-left: 0;
  }

  .tbl th, .tbl td {
    padding: 8px 10px;
    border-bottom: 1px solid #EEF2F7;
    vertical-align: middle;
    text-align: left;
  }

  .tbl thead th {
    background: #F3F7FB;
    color: #2C3E50;
    font-weight: 600;
    font-size: 14px;
  }

  .tbl tbody tr:hover {
    background: #FAFCFF;
  }
</style>
"""

display(HTML(html_message))

## Elimina dentro de la carpeta "resultados" archivos que no coinciden con el día de hoy

In [229]:
today_str = datetime.datetime.now(cst).strftime("%Y%m%d")

# 📁 Ruta de carpeta de resultados
results_dir = "resultados"

# 🧹 Crear carpeta si no existe
os.makedirs(results_dir, exist_ok=True)

# 🧽 Eliminar archivos cuyo nombre NO coincide con la fecha actual
for fname in os.listdir(results_dir):
    if not fname.endswith(".xlsx"):
        continue
    # Extraer la fecha del nombre del archivo (ej. cotizacion_20251025_153022.xlsx)
    try:
        date_part = fname.split("_")[1]
    except IndexError:
        continue
    # Si no coincide con el día actual, eliminar
    if not date_part.startswith(today_str):
        full_path = os.path.join(results_dir, fname)
        try:
            os.remove(full_path)
            print(f"🗑️ Eliminado archivo antiguo: {fname}")
        except Exception as e:
            print(f"⚠️ No se pudo eliminar {fname}: {e}")